In [1]:
import cv2 as cv
import numpy as np
import keyboard
import time

import Controlling_thymio
import global_planning
import vision
import loc_avoid
from ekf import ExtendedKalmanFilter
from ekf import apply_kalman

In [2]:
# Global constants here

IMAGE_WIDTH = 640 # width of the video feed

IMAGE_HEIGHT = 480 # height of the video feed

CAMERA_FPS = 30 # fps of the video feed

KEYBOARD_INPUT = "enter" # keyboard input to start

CAMERA_REFRESH_TIME = 30 # number of frame thrown away to allow the camera to focus in the meantime

SKIPPED_FRAME = 3 # get one frame every X

ACCEPTABLE_MOVEMENT = 200

In [3]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
import Controlling_thymio

node = await client.wait_for_node()

await node.lock()
aw(node.lock())

await node.wait_for_variables({"acc"})

In [4]:
def check_angle(angle):
    if  isinstance(angle, np.ndarray):
        return float(angle[0])
    else:
        return float(angle)


In [5]:
# 0 is laptop webcam, 1 is USB camera (!COMPUTER DEPENDENT!)
cap = cv.VideoCapture(0)

init = False # boolean set to True on keyboard input to start initialisation
is_init = False # boolean set to True once initialisation has succeeded
black_frame_remaining = 0 # numbers of frames to throw away
being_kidnapped = False # boolean set to True while the robot is being kidnapped
error_linear = [] # array of all linear errors
error_angle = [] # array of all angular errors
initial_turn = True # boolean set to True while the first orientation is not done
frame_number = 0 # to get one frame every X

iiiii = 0

# variable initialized at False irrelevant of wether it actually is
robot_not_found = False # boolean set to True if robot not found for one frame
camera_hidden = False # boolean set to True if robot not found for multiple frames

# create a robot instance
robot_instance = Controlling_thymio.Robot()

# if unable to connect to the camera
if not (cap.isOpened()):
    print("Could not open video device")
else:
    while(True):
        # get the input frame by frame (shape (480,640,3))
        ret, frame = cap.read() 
        cap.set(cv.CAP_PROP_FPS, CAMERA_FPS) 
        frame = cv.resize(frame, (IMAGE_WIDTH,IMAGE_HEIGHT), interpolation=cv.INTER_CUBIC) 

        iiiii += 1
        print(iiiii)

        # wait until keyboard input to initialize vision
        if keyboard.is_pressed(KEYBOARD_INPUT):
            init = True

        # reinitialize the robot after a kidnapping
        if being_kidnapped: 
            if not Controlling_thymio.kidnapping(node):
                
                # to give time to the kidnapper to get out of the frame
                time.sleep(3)
                
                # reinitialize the starting position of the robot
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                try:
                    start_x, start_y, robot_angle, width = vision.get_robot_position(fop)
                except:
                    break

                # reinitialize the global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x, objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle, width)
                obstacles_named = global_planning.naming_points(converted_obstacles, robot_instance, goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles, robot_instance, goal)
                path = global_planning.finding_path(adg_list, obstacles_named)

                black_frame_remaining = 0
                camera_hidden = False
                initial_turn = True
                is_init = True
                being_kidnapped = False
        
        # initialize 
        if init == True:
            try:
                # vision: start, obstacles and goal coordinates
                original_coordinates, new_coordinates = vision.get_fop_coordinates(frame)
                fop = vision.get_fop(frame, original_coordinates, new_coordinates)
                #start_x, start_y, robot_angle, width = vision.get_robot_position(fop)

                start_x, start_y, robot_angle, width = vision.get_robot_position(fop)

                #print(start_x,start_y)
                obstacles = vision.get_obstacles(fop, width)
                objective_x, objective_y = vision.get_objective(fop)

                # global planning
                converted_obstacles = [[tuple(arr) for arr in sublist] for sublist in obstacles]
                converted_obstacles = global_planning.obstacle_dictionnary(converted_obstacles)
                goal = (objective_x,objective_y)
                robot_instance.update_coordinates(start_x, start_y, robot_angle,width)
                obstacles_named = global_planning.naming_points(converted_obstacles,robot_instance,goal)
                adg_list = global_planning.creating_adjacency_dictionnary(converted_obstacles,robot_instance,goal)
                path = global_planning.finding_path(adg_list,obstacles_named)
                
                # extended kalman filter
                ekf = ExtendedKalmanFilter(([start_x,start_y,robot_angle,0,0]))
                ekf.set_time_t(time.time())
                left_wheel_speed = 0
                right_wheel_speed = 0

                black_frame_remaining = 0
                camera_hidden = False
                being_kidnapped = False
                is_init = True
            except Exception as e:
                print("Initialisation failed: " + str(e))

            init = False

        if is_init:  
        ################################## MOTION CONTROL HERE ################################## 
            
            # get out of the loop if the robot is being kidnapped
            if Controlling_thymio.kidnapping(node):
                being_kidnapped = True
                is_init = False
                continue

            fop = vision.get_fop(frame, original_coordinates, new_coordinates)

            # to use one frame every X
            #if frame_number%SKIPPED_FRAME == 0:

            if black_frame_remaining > 0:
                fop = cv.cvtColor(fop, cv.COLOR_BGR2GRAY)
                _, fop = cv.threshold(fop,255,255,cv.THRESH_BINARY)
                black_frame_remaining -= 1


            try:
                center_x, center_y, robot_angle, width = vision.get_robot_position(fop)
                if abs((center_x + center_y) - (robot_instance.center_x + robot_instance.center_y)) > ACCEPTABLE_MOVEMENT:
                    raise
                print("vision: ", center_x, center_y, robot_angle)
                robot_instance.update_coordinates(center_x, center_y, robot_angle, width)
                robot_not_found = False
            except:
                center_x = 0
                center_y = 0
                robot_angle = float(0)
                if robot_not_found:
                    camera_hidden = True
                robot_not_found = True

            #frame_number += 1
            
            # ekf update
            
            position = [center_x, center_y, check_angle(robot_angle)]
            speed = [float(left_wheel_speed),float(right_wheel_speed)]

            try:
                x = apply_kalman(ekf, position, speed)
                new_position = x[0:3]  # Extract the filtered position
                print('o')
            except ValueError as e:
                print(f"Error during Kalman filter update: {e}")
                new_position = [center_x, center_y, robot_angle]
            
            # camera was covered, now uncovered
            if center_x != 0 and center_y != 0 and camera_hidden:
                camera_hidden = False
                black_frame_remaining = 0 # to give it time to gain focus

                # position updated using Kalman
                print("Kalman: ", new_position[0], new_position[1], new_position[2])
                robot_instance.update_coordinates(new_position[0], new_position[1], new_position[2], width)

            # camera was uncovered, now covered
            elif center_x == 0 and center_y == 0:
                # position updated using Kalman PQQQQQQQQQQ?
                
                print("Kalman: ", new_position[0], new_position[1], new_position[2])
                robot_instance.update_coordinates(new_position[0], new_position[1], new_position[2], width)
            
            # camera was recently uncovered, set some frames to black to allow it time to gain focus
            elif black_frame_remaining > 0:
                # set the first frames after uncovering to black to allow time to focus -> navigation using extended Kalman filter

                # position updated using Kalman
                print("Kalman: ", new_position[0], new_position[1], new_position[2])
                robot_instance.update_coordinates(new_position[0], new_position[1], new_position[2], width)

            start_point = np.array([obstacles_named[path[0]][0],obstacles_named[path[0]][1]])
            end_point = np.array([obstacles_named[path[1]][0],obstacles_named[path[1]][1]])
            
            # traget point after the actual end point
            try:
                next_point = np.array([obstacles_named[path[2]][0],obstacles_named[path[2]][1]])
            except:
                next_point = [0, 0]

            # proximity avoidance
            if loc_avoid.check_local_nav(node, robot_instance, aw):
                left_wheel_speed, right_wheel_speed = loc_avoid.local_avoidance(node, robot_instance, aw)
                initial_turn = True
            # update of the wheel speed
            elif (position==np.array([0,0,0.0])).all():
                #left_wheel_speed, right_wheel_speed = 75,75
                
                #if letmecook : 
                #error_linear = np.append(error_linear,0.5)
                #error_angle = np.append(error_angle,0.5)
                left_wheel_speed, right_wheel_speed, error_linear, error_angle, initial_turn, path = Controlling_thymio.compute_wheel_speed(initial_turn, start_point, end_point, next_point, error_linear, error_angle, robot_instance, path, aw, node)
                #
                # letmecook = 0
                
                #left_wheel_speed, right_wheel_speed, error_linear, error_angle, initial_turn, path = Controlling_thymio.compute_wheel_speed(initial_turn, start_point, end_point, next_point, error_linear, error_angle, robot_instance, path, aw, node)   

                #robot_instance.update_speed(75,75)
                print("set speed low for camera hidden")
                
            else:
                left_wheel_speed, right_wheel_speed, error_linear, error_angle, initial_turn, path = Controlling_thymio.compute_wheel_speed(initial_turn, start_point, end_point, next_point, error_linear, error_angle, robot_instance, path, aw, node)   

            robot_instance.update_speed(left_wheel_speed, right_wheel_speed)

            print("Speed: ", left_wheel_speed, right_wheel_speed)
            # draw everything
            #if not camera_hidden and black_frame_remaining <= 0:
            cv.circle(fop, (robot_instance.center_x,robot_instance.center_y), radius=10, color=(0, 255, 0), thickness=-1)
            cv.circle(fop, (objective_x, objective_y), radius=10, color=(255, 0, 0), thickness=-1)
            cv.circle(fop, (start_x, start_y), radius=10, color=(0, 0, 255), thickness=-1)
            for obstacle in obstacles:
                for vertice in obstacle:
                    cv.circle(fop, (vertice[0],vertice[1]), radius=10, color=(0, 255, 0), thickness=-1)
            for i in range(len(path[:-1])):
                cv.line(fop,(obstacles_named[path[i]][0],obstacles_named[path[i]][1]),(obstacles_named[path[i+1]][0],obstacles_named[path[i+1]][1]),(255,0,0),5)
    
        #########################################################################################
        else:
            # to output the raw feed
            fop = frame

        cv.imshow('Raw',frame)
        cv.imshow('Processed',fop)

        await client.sleep(0.001)
        
        # Close all windows and get out of the loop if ESC is pressed
        if cv.waitKey(1) == 27:
            cv.destroyAllWindows()
            break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
